<a href="https://colab.research.google.com/github/priya170807/Time-seriesAnalysis/blob/main/atm_transformer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# !pip install torchsummary 

In [ ]:
import torch
import torch.nn as nn
import torchvision
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import MinMaxScaler
import os
import numpy as np
import pandas as pd
import math

In [ ]:
if torch.cuda.is_available():
  device = "cuda"
else:
  device = "cpu"
  

In [ ]:
device

'cuda'

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
import time
import numpy as np
import datetime
import argparse
import pickle
from io import StringIO

# encoding the timestamp data cyclically. See Medium Article.
def process_data(source, preprocess):

    dataset = pd.read_csv(source)
    df = dataset.copy()
    df['date'] = pd.to_datetime(dict(year=df.trans_year, month=df.trans_month, day=df.trans_date_set))
    df.set_index("date", inplace=True)
    timestamps_day = df["trans_date_set"]
    timestamps_month = df["trans_month"]
    
    # define the constants
    hours_in_day = 24
    days_in_month = 30
    month_in_year = 12
    
    # create new columns
    df['sin_day'] = np.sin(2*np.pi*timestamps_day/days_in_month)
    df['cos_day'] = np.cos(2*np.pi*timestamps_day/days_in_month)
    df['sin_month'] = np.sin(2*np.pi*timestamps_month/month_in_year)
    df['cos_month'] = np.cos(2*np.pi*timestamps_month/month_in_year)

    features = df.copy()
    
    # Removing the less impacting columns
    features.drop(["atm_name", "trans_date_set", "trans_month", "trans_year"], axis=1, inplace=True)

    # Select the numeric columns
    numeric_subset = ['increment', 'prevweek_mean', 'total_amount_withdrawn', 'sin_day', 'cos_day', 'sin_month', 'cos_month']
    numeric_df = features[numeric_subset]

    # Select the categorical columns
 
    categorical_subset = ['weekday','festival_religion', 'working_day', 'holiday_sequence']

    # One hot encode
    categorical_df = pd.get_dummies(features[categorical_subset])

    # Join the two dataframes using concat
    # Make sure to use axis = 1 to perform a column bind
    features = pd.concat([numeric_df, categorical_df], axis = 1)
    
    # create X and y.
    X = features.drop("total_amount_withdrawn", axis=1).copy()
    y = features["total_amount_withdrawn"]
    
    # split the data into training and test sets.
    split_size = int(len(features)*0.8)
    Xtrain = X[:split_size]
    Xtest = X[split_size:]
    ytrain = y[:split_size]
    ytest = y[split_size:]

     # print values to understand shapes
    print("the split_size={}".format(split_size))
    print("the Xtrain_data length={}".format(len(Xtrain)))
    print("the Xtest_data length={}".format(len(Xtest)))
    print("the ytrain_data length={}".format(len(ytrain)))
    print("the ytest_data length={}".format(len(ytest)))
    
    # Preprocess the data (if applicable)
    if preprocess:
        
        scaler_x = MinMaxScaler()
        scaler_y = MinMaxScaler()
        scale_subset = ['increment', 'prevweek_mean']
        Xtrain.loc[:, scale_subset] = scaler_x.fit_transform(Xtrain.loc[:, scale_subset])
        Xtest.loc[:, scale_subset] = scaler_x.transform(Xtest.loc[:, scale_subset])
        ytrain_scaled = scaler_y.fit_transform(ytrain.values.reshape(-1,1))
        ytest_scaled = scaler_y.transform(ytest.values.reshape(-1,1))
        ytrain_df = pd.Series(ytrain_scaled.flatten(), index=Xtrain.index.tolist())
        ytest_df = pd.Series(ytest_scaled.flatten(), index=Xtest.index.tolist())
        train_data = Xtrain.copy()
        test_data = Xtest.copy()
        train_data["total_amount_withdrawn"] = ytrain_df
        test_data["total_amount_withdrawn"] = ytest_df
        print("the train_data shape = {}".format(train_data.shape))
        print("the test_data shape = {}".format(test_data.shape))
        print(train_data.head())
    
        scaler_filename = '/content/drive/MyDrive/atm_usecase/scaler.pkl'
        with open(scaler_filename, 'wb') as scaler_file:
            pickle.dump(scaler_y, scaler_file)
        # gcs_scaler_path = 'gs://{0}/{1}/{2}'.format(dest_bucket_name, dest_prefix, scaler_filename)
        # subprocess.check_call(['gsutil', 'cp', scaler_filename, gcs_scaler_path])
        # print('Saved scaler pickle file in: {}'.format(gcs_scaler_path))


    else:
        Xtrain = train_data.to_numpy()
        Xtest = test_data.to_numpy()

    scaled_train_data = train_data
    scaled_test_data = test_data
    # print("train_data after scaling")
    # print(scaled_train_data.head())

    scaled_train_data.to_csv(r'/content/drive/MyDrive/datasets/atm_train_dataset.csv')
    scaled_test_data.to_csv(r'/content/drive/MyDrive/datasets/atm_test_dataset.csv')

# if __name__ == '__main__':
#     parser = argparse.ArgumentParser(description="preprocessing the original file")
#     parser.add_argument('--source', type=str, default="")
#     parser.add_argument('--preprocess', type=bool, default=True)
#     args = parser.parse_args()
#     preprocess_data(source=args.source,
#                     preprocess=args.preprocess)


# train_dataset = process_data('/content/drive/MyDrive/atm_data.csv')
# train_dataset.to_csv(r'/content/drive/MyDrive/clean_atm_dataset.csv', index=False)


In [ ]:
source="/content/drive/MyDrive/datasets/atm_data.csv"
preprocess=True

In [ ]:
process_data(source, preprocess)

the split_size=1795
the Xtrain_data length=1795
the Xtest_data length=449
the ytrain_data length=1795
the ytest_data length=449
the train_data shape = (1795, 29)
the test_data shape = (449, 29)
            increment  ...  total_amount_withdrawn
date                   ...                        
2011-01-01   0.000000  ...                0.635822
2011-01-01   0.000543  ...                0.585407
2011-01-01   0.001086  ...                0.534638
2011-01-02   0.001630  ...                0.591222
2011-01-02   0.002173  ...                0.407644

[5 rows x 29 columns]


/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1734: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value[:, i].tolist())
/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1734: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value[:, i].tolist())


In [ ]:
csv_file='/content/drive/MyDrive/datasets/atm_test_dataset.csv'
df = pd.read_csv(csv_file)

In [ ]:
df.head()

,date,increment,prevweek_mean,sin_day,cos_day,sin_month,cos_month,weekday_FRIDAY,weekday_MONDAY,weekday_SATURDAY,weekday_SUNDAY,weekday_THURSDAY,weekday_TUESDAY,weekday_WEDNESDAY,festival_religion_C,festival_religion_H,festival_religion_M,festival_religion_N,festival_religion_NH,working_day_H,working_day_W,holiday_sequence_HHH,holiday_sequence_HHW,holiday_sequence_HWH,holiday_sequence_HWW,holiday_sequence_WHH,holiday_sequence_WHW,holiday_sequence_WWH,holiday_sequence_WWW,total_amount_withdrawn
0,2016-02-05,1.000543,0.290489,0.866025,0.5,0.866025,0.5,0,0,0,1,0,0,0,0,0,0,0,1,1,0,0,1,0,0,0,0,0,0,0.248245
1,2016-02-05,1.000978,0.203825,0.866025,0.5,0.866025,0.5,0,1,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,1,0,0,0,0,0.012480
2,2016-02-05,1.001521,0.203825,0.866025,0.5,0.866025,0.5,0,0,0,0,0,1,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0.165355
3,2016-02-05,1.002064,0.203825,0.866025,0.5,0.866025,0.5,0,0,0,0,0,0,1,0,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0.019854
4,2016-02-05,1.002607,0.203825,0.866025,0.5,0.866025,0.5,0,0,0,0,1,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0.141459


In [ ]:
df.shape

(449, 30)

In [ ]:
df.head().values[:,-1]

array([0.24824505424377796, 0.012479614266468129, 0.1653548890307027,
       0.01985393178756293, 0.1414592639863859], dtype=object)

In [ ]:
class ATMDataset(Dataset):
    """Face Landmarks dataset."""

    def __init__(self, csv_name, root_dir, training_length, forecast_window):
        """
        Args:
            csv_file (string): Path to the csv file.
            root_dir (string): Directory
        """
        
        # load raw data file
        csv_file = os.path.join(root_dir, csv_name)
        self.df = pd.read_csv(csv_file)
        self.df.set_index("date", inplace=True)
        print(len(self.df))
        self.X = self.df.values
        # self.y = self.df["total_amount_withdrawn"].values
        self.root_dir = root_dir
        self.T = training_length
        self.S = forecast_window

    def __len__(self):
        # return number of sensors
        return len(self.df) - self.T - self.S

    # Will pull an index between 0 and __len__. 
    def __getitem__(self, idx):
        
        # print("inside __get_item__")
        # print('idx={}'.format(idx))
        # print('self.T={}'.format(self.T))
        # print('self.S={}'.format(self.S))
        
       
        index_in = torch.tensor([i for i in range(idx, idx+self.T)])
        index_tar = torch.tensor([i for i in range(idx + self.T, idx + self.T + self.S)])
        # print("index_in")
        # print(index_in)
        # print('index_tar')
        # print(index_tar)
        _input = torch.from_numpy(self.X[idx : idx + self.T])
        target = torch.from_numpy(self.X[idx + self.T : idx + self.T + self.S])

        return index_in, index_tar, _input, target

In [ ]:
csv_name='atm_train_dataset.csv'
csv_name_test='atm_test_dataset.csv'
root_dir='/content/drive/MyDrive/datasets/'
training_length=48
forecast_window=24

In [ ]:
train_dataset = ATMDataset(csv_name = csv_name, root_dir = root_dir, training_length = training_length, forecast_window = forecast_window)
test_dataset = ATMDataset(csv_name = csv_name_test, root_dir = root_dir, training_length = training_length, forecast_window = forecast_window)

1795
449


In [ ]:
train_dataloader = DataLoader(train_dataset, batch_size=1, shuffle=False)
test_dataloader = DataLoader(test_dataset, batch_size=1, shuffle=False)

In [ ]:
# train_dataset[1000]

In [ ]:
len(train_dataset), len(test_dataloader)

(1723, 377)

In [ ]:
# for index, data in enumerate(test_dataloader):
  # print(index)
  # print("/n")
  # print(data)
  # print("/....................................................../")

In [ ]:
# shape of data returned by the dataset_loader = ((1, 48, 29), (1, 24, 29))

Define the required constants

In [ ]:
# define the required constants
epoch = 50
# k = 60
batch_size = 1
# frequency = 100
training_length = 48
forecast_window = 24
root_dir = '/content/drive/MyDrive/datasets/'
train_csv = "atm_train_dataset.csv"
test_csv = "atm_test_dataset.csv"
path_to_save_model = "/content/drive/MyDrive/atm_usecase/"
path_to_save_loss = "/content/drive/MyDrive/atm_usecase/"
path_to_save_predictions = "/content/drive/MyDrive/atm_usecase/"
device = "cuda"

Model code

In [ ]:
import torch.nn as nn
import torch, math
# from icecream import ic
import time
"""
The architecture is based on the paper “Attention Is All You Need”. 
Ashish Vaswani, Noam Shazeer, Niki Parmar, Jakob Uszkoreit, Llion Jones, Aidan N Gomez, Lukasz Kaiser, and Illia Polosukhin. 2017.
"""

class Transformer(nn.Module):
    # d_model : number of features
    def __init__(self,feature_size=29,num_layers=3,dropout=0):
        super(Transformer, self).__init__()

        self.encoder_layer = nn.TransformerEncoderLayer(d_model=feature_size, nhead=1, dropout=dropout)
        self.transformer_encoder = nn.TransformerEncoder(self.encoder_layer, num_layers=num_layers)        
        self.decoder = nn.Linear(feature_size,1)
        self.init_weights()

    def init_weights(self):
        initrange = 0.1    
        self.decoder.bias.data.zero_()
        self.decoder.weight.data.uniform_(-initrange, initrange)

    def _generate_square_subsequent_mask(self, sz):
        mask = (torch.triu(torch.ones(sz, sz)) == 1).transpose(0, 1)
        mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))
        return mask

    def forward(self, src, device):
        
        mask = self._generate_square_subsequent_mask(len(src)).to(device)
        output = self.transformer_encoder(src,mask)
        output = self.decoder(output)
        return output
        



Use the teacher-forcing model for the training purpose

In [ ]:
# the log-loss calculator

import os, shutil

# save train or validation loss
def log_loss(loss_val : float, path_to_save_loss : str, train : bool = True):
    if train:
        file_name = "train_loss.txt"
    else:
        file_name = "val_loss.txt"

    path_to_file = path_to_save_loss+file_name
    os.makedirs(os.path.dirname(path_to_file), exist_ok=True)
    with open(path_to_file, "a") as f:
        f.write(str(loss_val)+"\n")
        f.close()


In [ ]:
# Plots the trainig loss for every ten epochs
import matplotlib.pyplot as plt
import numpy as np
import torch

def plot_training(epoch, path_to_save, src, prediction, index_in, index_tar):

    # idx_scr = index_in.tolist()[0]
    # idx_tar = index_tar.tolist()[0]
    # idx_pred = idx_scr.append(idx_tar.append([idx_tar[-1] + 1]))

    idx_scr = [i for i in range(len(src))]
    idx_pred = [i for i in range(1, len(prediction)+1)]

    plt.figure(figsize=(15,6))
    plt.rcParams.update({"font.size" : 18})
    plt.grid(b=True, which='major', linestyle = '-')
    plt.grid(b=True, which='minor', linestyle = '--', alpha=0.5)
    plt.minorticks_on()

    plt.plot(idx_scr, src, 'o-.', color = 'blue', label = 'input sequence', linewidth=1)
    plt.plot(idx_pred, prediction, 'o-.', color = 'limegreen', label = 'prediction sequence', linewidth=1)

    plt.title("Teaching Forcing from Epoch " + str(epoch))
    plt.xlabel("Time Elapsed")
    plt.ylabel("Humidity (%)")
    plt.legend()
    plt.savefig(path_to_save+f"/Epoch_{str(epoch)}.png")
    plt.close()


In [ ]:
# from model import Transformer
from torch.utils.data import DataLoader
import torch
import torch.nn as nn
# from DataLoader import SensorDataset
import logging
import time # debugging
# from plot import *
# from helpers import *
from joblib import load
# from icecream import ic
from torch.optim.lr_scheduler import ReduceLROnPlateau

logging.basicConfig(level=logging.INFO, format="%(asctime)s [%(levelname)s] %(name)s %(message)s", datefmt="[%Y-%m-%d %H:%M:%S]")
logger = logging.getLogger(__name__)

def transformer(dataloader, EPOCH, path_to_save_model, path_to_save_loss, path_to_save_predictions, device):

    device = torch.device(device)

    model = Transformer().double().to(device)
    optimizer = torch.optim.Adam(model.parameters())
    # scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=200)
    criterion = torch.nn.MSELoss()
    best_model = ""
    min_train_loss = float('inf')

    for epoch in range(EPOCH + 1):
        print("/n")
        print("/n")
        print("*****************************************************************************************************")
        print(f"The epoch started / epoch number = {epoch}")
        train_loss = 0
        val_loss = 0

        ## TRAIN -- TEACHER FORCING
        model.train()
        for index_in, index_tar, _input, target in dataloader: # for each data set 
        
            optimizer.zero_grad()

            # Shape of _input : [batch, sequence_length, feature]
            # Desired input for model: [sequence_length, batch, feature]
            # print(f'_input shape = {_input.shape}')   # torch.Size([1,48,29])
            # print(f'target shape = {target.shape}')   # torch.Size([1,24,29])
            src = _input.permute(1,0,2).double().to(device)[:-1,:,:] # torch.Size([47, 1, 29]) all except the current timestamp
            # print(f'The shape of src after permute = {src.shape}')
            target = _input.permute(1,0,2).double().to(device)[1:,:,:] # src shifted by 1.
            # print(f'The shape of target after permute = {target.shape}')
            prediction = model(src, device) 
            # print(f'prediction shape={prediction.shape}')
            loss = criterion(prediction, target[:,:,-1].unsqueeze(-1))
            loss.backward()
            optimizer.step()
            # scheduler.step(loss.detach().item())
            train_loss += loss.detach().item()

        if train_loss < min_train_loss:
            torch.save(model.state_dict(), path_to_save_model + f"best_train_{epoch}.pth")
            torch.save(optimizer.state_dict(), path_to_save_model + f"optimizer_{epoch}.pth")
            min_train_loss = train_loss
            best_model = f"best_train_{epoch}.pth"


        if epoch % 10 == 0: # Plot 1-Step Predictions

            print(f'The epoch number = {epoch}')

            logger.info(f"Epoch: {epoch}, Training loss: {train_loss}")
            scaler = load('/content/drive/MyDrive/atm_usecase/scaler.pkl')
            src_amount = scaler.inverse_transform(src[:,:,-1].cpu()) #torch.Size([35, 1, 7])
            print(f'src_amount shape = {src_amount.shape}')
            print(f'src_amount values = {src_amount[:5]}')
            target_amount = scaler.inverse_transform(target[:,:,-1].cpu()) #torch.Size([35, 1, 7])
            print(f'target_amount shape = {target_amount.shape}')
            print(f'target_amount values = {target_amount[:5]}')
            prediction_amount = scaler.inverse_transform(prediction[:,:,-1].detach().cpu().numpy()) #torch.Size([35, 1, 7])
            print(f'prediction_amount shape = {prediction_amount.shape}')
            print(f'prediction_amount values = {prediction_amount[:5]}')
            plot_training(epoch, path_to_save_predictions, src_amount, prediction_amount, index_in, index_tar)

        train_loss /= len(dataloader)
        log_loss(train_loss, path_to_save_loss, train=True)
        
    # plot_loss(path_to_save_loss, train=True)
    return best_model

Call the transformer model

In [ ]:
 best_model = transformer(train_dataloader, epoch, path_to_save_model, path_to_save_loss, path_to_save_predictions, device)

/n
/n
*****************************************************************************************************
The epoch started / epoch number = 0


[2021-06-24 05:13:04] [INFO] __main__ Epoch: 0, Training loss: 19.003887664689042


The epoch number = 0
src_amount shape = (47, 1)
src_amount values = [[771900.]
 [488900.]
 [833200.]
 [476800.]
 [686100.]]
target_amount shape = (47, 1)
target_amount values = [[488900.]
 [833200.]
 [476800.]
 [686100.]
 [512800.]]
prediction_amount shape = (47, 1)
prediction_amount values = [[280268.86569467]
 [429866.20087738]
 [471846.47690701]
 [688326.94559947]
 [730487.06080554]]
/n
/n
*****************************************************************************************************
The epoch started / epoch number = 1
/n
/n
*****************************************************************************************************
The epoch started / epoch number = 2
/n
/n
*****************************************************************************************************
The epoch started / epoch number = 3
/n
/n
*****************************************************************************************************
The epoch started / epoch number = 4
/n
/n
************************

[2021-06-24 05:15:11] [INFO] __main__ Epoch: 10, Training loss: 5.700125970115612


The epoch number = 10
src_amount shape = (47, 1)
src_amount values = [[771900.]
 [488900.]
 [833200.]
 [476800.]
 [686100.]]
target_amount shape = (47, 1)
target_amount values = [[488900.]
 [833200.]
 [476800.]
 [686100.]
 [512800.]]
prediction_amount shape = (47, 1)
prediction_amount values = [[513490.2570361 ]
 [791207.7984417 ]
 [488052.75300836]
 [690448.59192544]
 [527923.73385492]]
/n
/n
*****************************************************************************************************
The epoch started / epoch number = 11
/n
/n
*****************************************************************************************************
The epoch started / epoch number = 12
/n
/n
*****************************************************************************************************
The epoch started / epoch number = 13
/n
/n
*****************************************************************************************************
The epoch started / epoch number = 14
/n
/n
*******************

...

The epoch number = 30
src_amount shape = (47, 1)
src_amount values = [[771900.]
 [488900.]
 [833200.]
 [476800.]
 [686100.]]
target_amount shape = (47, 1)
target_amount values = [[488900.]
 [833200.]
 [476800.]
 [686100.]
 [512800.]]
prediction_amount shape = (47, 1)
prediction_amount values = [[492547.80391576]
 [827767.78640974]
 [485461.84891201]
 [705982.74147961]
 [519787.08859523]]
/n
/n
*****************************************************************************************************
The epoch started / epoch number = 31
/n
/n
*****************************************************************************************************
The epoch started / epoch number = 32
/n
/n
*****************************************************************************************************
The epoch started / epoch number = 33
/n
/n
*****************************************************************************************************
The epoch started / epoch number = 34
/n
/n
*******************

[2021-06-24 05:21:35] [INFO] __main__ Epoch: 40, Training loss: 0.5098468464801569


The epoch number = 40
src_amount shape = (47, 1)
src_amount values = [[771900.]
 [488900.]
 [833200.]
 [476800.]
 [686100.]]
target_amount shape = (47, 1)
target_amount values = [[488900.]
 [833200.]
 [476800.]
 [686100.]
 [512800.]]
prediction_amount shape = (47, 1)
prediction_amount values = [[486278.86460174]
 [831890.94553103]
 [486165.93151698]
 [699561.6420692 ]
 [515157.26259593]]
/n
/n
*****************************************************************************************************
The epoch started / epoch number = 41
/n
/n
*****************************************************************************************************
The epoch started / epoch number = 42
/n
/n
*****************************************************************************************************
The epoch started / epoch number = 43
/n
/n
*****************************************************************************************************
The epoch started / epoch number = 44
/n
/n
*******************

[2021-06-24 05:23:43] [INFO] __main__ Epoch: 50, Training loss: 0.4331184482470817


The epoch number = 50
src_amount shape = (47, 1)
src_amount values = [[771900.]
 [488900.]
 [833200.]
 [476800.]
 [686100.]]
target_amount shape = (47, 1)
target_amount values = [[488900.]
 [833200.]
 [476800.]
 [686100.]
 [512800.]]
prediction_amount shape = (47, 1)
prediction_amount values = [[479442.96873017]
 [830367.69227619]
 [476997.82906093]
 [708825.76660269]
 [504352.84809719]]


In [ ]:
def plot_prediction(title, path_to_save, src, tgt, prediction, index_in, index_tar):

    idx_scr = index_in[0, 1:].tolist()
    idx_tgt = index_tar[0].tolist()
    idx_pred = [i for i in range(idx_scr[0] +1, idx_tgt[-1])] #t2 - t61

    plt.figure(figsize=(15,6))
    plt.rcParams.update({"font.size" : 16})

    # connect with last elemenet in src
    # tgt = np.append(src[-1], tgt.flatten())
    # prediction = np.append(src[-1], prediction.flatten())

    # plotting
    plt.plot(idx_scr, src, '-', color = 'blue', label = 'Input', linewidth=2)
    plt.plot(idx_tgt, tgt, '-', color = 'indigo', label = 'Target', linewidth=2)
    plt.plot(idx_pred, prediction,'--', color = 'limegreen', label = 'Forecast', linewidth=2)

    #formatting
    plt.grid(b=True, which='major', linestyle = 'solid')
    plt.minorticks_on()
    plt.grid(b=True, which='minor', linestyle = 'dashed', alpha=0.5)
    plt.xlabel("Time Elapsed")
    plt.ylabel("Total amount withdrawn (%)")
    plt.legend()
    plt.title("Forecast of an ATM's cash deamnd")

    # save
    plt.savefig(path_to_save+f"Prediction_{title}.png")
    plt.close()

In [ ]:
# from model import Transformer
from torch.utils.data import DataLoader
import torch
import torch.nn as nn
# from DataLoader import SensorDataset
import logging
import time # debugging
# from plot import *
# from helpers import *
from joblib import load
# from icecream import ic
from torchvision import models
from torchsummary import summary


logging.basicConfig(level=logging.INFO, format="%(asctime)s [%(levelname)s] %(name)s %(message)s", datefmt="[%Y-%m-%d %H:%M:%S]")
logger = logging.getLogger(__name__)

def inference(path_to_save_predictions, forecast_window, dataloader, device, path_to_save_model, best_model):

    print("Inside inference method")
    device = torch.device(device)
    


    model = Transformer().double().to(device)
    # summary(model, input_size=(47,1,29), batch_size=1, device=device)
    model.load_state_dict(torch.load(path_to_save_model+best_model))
    print(model)
    criterion = torch.nn.MSELoss()

    val_loss = 0
    with torch.no_grad():

        model.eval()
        for plot in range(2):
          
          for index_in, index_tar, _input, target in dataloader:
            # index_in, index_tar, _input, target = next(iter(test_dataloader))
                
            # print(f'index_in={index_in}')
            # print(f'index_tar={index_tar}')
            # print(f'the _input shape = {_input.shape}')
            # print(f'the target shape = {target.shape}')
            # starting from 1 so that src matches with target, but has same length as when training
            src = _input.permute(1,0,2).double().to(device)[1:, :, :] # 47, 1, 29: t1 -- t47
            # print(f'the src shape initially = {src.shape}')
            target = target.permute(1,0,2).double().to(device) # t48 - t59 [24,1,29]
            # print(f'target shape  initially= {target.shape}')
            # print(f'target 49 index = {target[1,0,:]}')
            next_input_model = src
            all_predictions = []

            for i in range(forecast_window-1):
                
                prediction = model(next_input_model, device) # 47,1,1: t2' - t48'
                # print(f'prediction shape for every iteration in forecast window={prediction.shape}')
                if all_predictions == []:
                    all_predictions = prediction # 47,1,1: t2' - t48'
                else:
                    all_predictions = torch.cat((all_predictions, prediction[-1,:,:].unsqueeze(0))) # 47+,1,1: t2' - t48', t49', t50'
                print(f'all_predictions shape = {all_predictions.shape}')
                pos_encoding_old_vals = src[i+1:, :, :-1]
                # print(f'shape of pos_encoding_old_vals = {pos_encoding_old_vals.shape}') # 46, 1, 6, pop positional encoding first value: t2 -- t47
                pos_encoding_new_val = target[i, :, :-1].unsqueeze(1) # 1, 1, 6, append positional encoding of last predicted value: t48
                # print(f'shape of pos_encoding_new_val = {pos_encoding_new_val.shape}')
                # print(f'target new value = {pos_encoding_new_val}')
                pos_encodings = torch.cat((pos_encoding_old_vals, pos_encoding_new_val)) # 47, 1, 6 positional encodings matched with prediction: t2 -- t48
                # print(f'shape of pos_encodings = {pos_encodings.shape}')
                
                # print(f"prediction[-1,:,:].shape = {prediction[-1,:,:].shape}")
                # print(f'src[i+1:,:,-1].shape = {src[i+1:,:,-1].shape}')
                next_input_model = torch.cat((src[i+1:, :, -1].unsqueeze(-1), prediction[-1,:,:].unsqueeze(0))) #t2 -- t47, t48'
                # print(f'shape of next_input_model = {next_input_model.shape}')
                next_input_model = torch.cat((next_input_model, pos_encodings), dim = 2) # 47, 1, 7 input for next round
            # print(f'src[1:,:,-1].shape={src[1:,:,-1].shape}')
            # print(f'target[:-1,:,-1].shape={target[:-1,:,-1].shape}')
            true = torch.cat((src[1:,:,-1],target[:-1,:,-1]))
            loss = criterion(true, all_predictions[:,:,-1])
            val_loss += loss
          
        val_loss = val_loss/all_predictions.shape[0]
        log_loss(val_loss, path_to_save_loss, train=False)
        scaler = load('/content/drive/MyDrive/atm_usecase/scaler.pkl')
        src_amount = scaler.inverse_transform(src[:,:,-1].cpu())
        target_amount = scaler.inverse_transform(target[:,:,-1].cpu())
        prediction_amount = scaler.inverse_transform(all_predictions[:,:,-1].detach().cpu().numpy())
        plot_prediction(plot, path_to_save_predictions, src_amount, target_amount, prediction_amount, index_in, index_tar)

    logger.info(f"Loss On Unseen Dataset: {val_loss.item()}")

In [ ]:
inference(path_to_save_predictions, forecast_window, test_dataloader, device, path_to_save_model, best_model)

Streaming output truncated to the last 5000 lines.
all_predictions shape = torch.Size([61, 1, 1])
all_predictions shape = torch.Size([62, 1, 1])
all_predictions shape = torch.Size([63, 1, 1])
all_predictions shape = torch.Size([64, 1, 1])
all_predictions shape = torch.Size([65, 1, 1])
all_predictions shape = torch.Size([66, 1, 1])
all_predictions shape = torch.Size([67, 1, 1])
all_predictions shape = torch.Size([68, 1, 1])
all_predictions shape = torch.Size([69, 1, 1])
all_predictions shape = torch.Size([47, 1, 1])
all_predictions shape = torch.Size([48, 1, 1])
all_predictions shape = torch.Size([49, 1, 1])
all_predictions shape = torch.Size([50, 1, 1])
all_predictions shape = torch.Size([51, 1, 1])
all_predictions shape = torch.Size([52, 1, 1])
all_predictions shape = torch.Size([53, 1, 1])
all_predictions shape = torch.Size([54, 1, 1])
all_predictions shape = torch.Size([55, 1, 1])
all_predictions shape = torch.Size([56, 1, 1])
all_predictions shape = torch.Size([57, 1, 1])
all_predi

[2021-06-24 06:24:51] [INFO] __main__ Loss On Unseen Dataset: 0.6147503675319522


In [ ]:
# import itertools
# index_in, index_tar, _input, target = next(iter(test_dataloader))